In [5]:
import cv2
import depthai as dai
import numpy as np
import os

def find_homography(src_pts, dst_pts):
  """
  Estimate homography matrix using RANSAC algorithm.
  """
  assert len(src_pts) >= 4 and len(dst_pts) >= 4, "At least 4 corresponding points are required."

  # Compute homography matrix
  homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

  # Ensure homography matrix is of the correct type and size
  if homography is not None and homography.shape == (3, 3):
    if homography.dtype != np.float32:
      homography = homography.astype(np.float32)
    return homography
  else:
    return None

def frame_to_np(frame):
  # Convert the frame to BGR format
  frame = frame.getCvFrame()

  # Convert to grayscale if needed
  if len(frame.shape) == 3:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

  return frame

def save_frames(frames, folder):
  if not os.path.exists(folder):
    os.makedirs(folder)

  for i, frame in enumerate(frames):
    cv2.imwrite(os.path.join(folder, f"frame_{i}.jpg"), frame)

# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a color camera node
cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(640, 480)
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create an XLinkOut to send the video frames
xout = pipeline.createXLinkOut()
xout.setStreamName("video")

# Link the camera to the XLinkOut
cam_rgb.preview.link(xout.input)

# Pipeline defined, now we can connect to the device
with dai.Device(pipeline) as device:
  # Start pipeline
  device.startPipeline()

  # Output queue will be used to get the frames from the color camera
  q = device.getOutputQueue(name="video", maxSize=4, blocking=False)

  # Initialize variables
  frames = []
  start_time = None

  # Get current time
  start_time = cv2.getTickCount()

  # Main loop to capture frames for 5 seconds
  while (cv2.getTickCount() - start_time) / cv2.getTickFrequency() < 5:
    # Get the frame from the queue
    in_frame = q.get()
    frame = frame_to_np(in_frame)
    frames.append(frame)

  # Save the frames to a folder
  save_frames(frames, "panorama_images")

# Stitch the images from the folder
images_folder = "panorama_images"
images = [cv2.imread(os.path.join(images_folder, f)) for f in os.listdir(images_folder) if f.endswith('.jpg')]

stitched_image = None
prev_frame = None

# Loop through the images and perform stitching
for frame in images:
  if prev_frame is not None:
    # Image Stitching
    # Step 1: Detect features and compute descriptors using SIFT
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(prev_frame, None)
    keypoints2, descriptors2 = sift.detectAndCompute(frame, None)

    # Step 2: Match features between the two images using BFMatcher
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test to filter good matches
    good_matches = []
    for m, n in matches:
      if m.distance < 0.75 * n.distance:
        good_matches.append(m)

    # Convert keypoints to numpy arrays
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Step 3: Estimate homography matrix based on good matches
    homography = find_homography(src_pts, dst_pts)

    # Step 4: Warp the previous frame to the current frame with considering overlap
    frame_width, frame_height = frame.shape[:2]
    result_width = frame_width + prev_frame.shape[1]  # Assume some overlap
    result_height = max(frame_height, prev_frame.shape[0])  # Take max height

    result = cv2.warpPerspective(prev_frame, homography, (result_width, result_height))

    # Step 5: Blend the current frame onto the warped previous frame (considering overlap)
    mask = cv2.warpPerspective(np.ones(prev_frame.shape[:2], dtype=np.float32), homography, (result_width, result_height))

    # Blend using alpha blending for seamless transition in overlap region
    alpha = 0.5  # Adjust alpha for blending intensity (0 - transparent, 1 - opaque)
    result = cv2.addWeighted(result, 1 - mask, frame, alpha, 0)

    # Update stitched image and previous frame
    stitched_image = result
    prev_frame = frame

# Display the stitched image
cv2.imshow("Stitched Panorama", stitched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


RuntimeError: No available devices

In [2]:
import cv2
import depthai as dai
import numpy as np
import os

def find_homography(src_pts, dst_pts):
    """
    Estimate homography matrix using RANSAC algorithm.
    """
    assert len(src_pts) >= 4 and len(dst_pts) >= 4, "At least 4 corresponding points are required."

    # Compute homography matrix
    homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

    # Ensure homography matrix is of the correct type and size
    if homography is not None and homography.shape == (3, 3):
        if homography.dtype != np.float32:
            homography = homography.astype(np.float32)
        return homography
    else:
        return None

# Function to convert a DepthAI frame to a numpy array
def frame_to_np(frame):
    # Convert the frame to BGR format
    frame = frame.getCvFrame()

    # Convert to grayscale if needed
    if len(frame.shape) == 3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    return frame

# Function to save frames to a folder
def save_frames(frames, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i, frame in enumerate(frames):
        cv2.imwrite(os.path.join(folder, f"frame_{i}.jpg"), frame)

# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a color camera node
cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(640, 480)
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create an XLinkOut to send the video frames
xout = pipeline.createXLinkOut()
xout.setStreamName("video")

# Link the camera to the XLinkOut
cam_rgb.preview.link(xout.input)

# Pipeline defined, now we can connect to the device
with dai.Device(pipeline) as device:
    # Start pipeline
    device.startPipeline()

    # Output queue will be used to get the frames from the color camera
    q = device.getOutputQueue(name="video", maxSize=4, blocking=False)

    # Initialize variables
    frames = []
    start_time = None

    # Get current time
    start_time = cv2.getTickCount()

    # Main loop to capture frames for 5 seconds
    while (cv2.getTickCount() - start_time) / cv2.getTickFrequency() < 5:
        # Get the frame from the queue
        in_frame = q.get()
        frame = frame_to_np(in_frame)
        frames.append(frame)

    # Save the frames to a folder
    save_frames(frames, "panorama_images")

# Stitch the images from the folder
images_folder = "panorama_images"
images = [cv2.imread(os.path.join(images_folder, f)) for f in os.listdir(images_folder) if f.endswith('.jpg')]

# Initialize variables
stitched_image = None
prev_frame = None

# Loop through the images and perform stitching
for frame in images:
    # If previous frame exists, stitch the current frame with the previous one
    if prev_frame is not None:
        # Perform image stitching
        # Step 1: Detect features and compute descriptors using SIFT
        sift = cv2.SIFT_create()
        keypoints1, descriptors1 = sift.detectAndCompute(prev_frame, None)
        keypoints2, descriptors2 = sift.detectAndCompute(frame, None)

        # Step 2: Match features between the two images using BFMatcher
        matcher = cv2.BFMatcher()
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

        # Apply ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        # Convert keypoints to numpy arrays
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Step 3: Estimate homography matrix
        homography = find_homography(src_pts, dst_pts)

        # Step 4: Warp the previous frame to the current frame
        height, width = prev_frame.shape[:2]
        result = cv2.warpPerspective(prev_frame, homography, (width * 2, height))

        # Step 5: Copy the current frame to the result image
        result[:frame.shape[0], :frame.shape[1]] = frame

        # Update stitched image
        stitched_image = result

    # Update previous frame
    prev_frame = frame

# Display the stitched image
plt.imshow(stitched_image)
plt.axis('off')
plt.show()


/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_47922/2046785680.py:61: DeprecationWarning: Device(pipeline) starts the pipeline automatically. Use Device() and startPipeline(pipeline) otherwise
  device.startPipeline()


AssertionError: At least 4 corresponding points are required.

In [3]:
import numpy as np
import cv2
import depthai as dai

# Initialize variables
stitched_image = None
prev_frame = None
start_time = None

# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a stereo camera node
stereo = pipeline.createStereoDepth()
stereo.setOutputDepth(True)
stereo.setConfidenceThreshold(200)

# Define a color camera node
left = pipeline.createXLinkIn()
right = pipeline.createXLinkIn()
left.setStreamName("left")
right.setStreamName("right")

# Create output
xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
xout.input.setBlocking(False)

# Connect the stereo camera
left.out.link(stereo.left)
right.out.link(stereo.right)

# Connect the disparity output
stereo.disparity.link(xout.input)

# Pipeline defined, now we can connect to the device
with dai.Device(pipeline) as device:
    # Start pipeline
    device.startPipeline()

    # Output queue will be used to get the disparity frames from the stereo camera
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)

    # Get current time
    start_time = cv2.getTickCount()

    # Main loop to capture frames for 5 seconds and stitch them
    while (cv2.getTickCount() - start_time) / cv2.getTickFrequency() < 5:
        # Get the stereo frames
        in_depth = q.get()
        frame = in_depth.getFrame()

        # Convert disparity frame to BGR format for visualization
        frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # If previous frame exists, stitch the current frame with the previous one
        if prev_frame is not None:
            # Stitch frames
            stitched_image = np.hstack([prev_frame, frame])

            # Display the stitched image
            cv2.imshow('Stitched Image', stitched_image)
            cv2.waitKey(1)  # Show the image for a short duration

        # Update previous frame
        prev_frame = frame

    # Close the OpenCV windows after displaying for 5 seconds
    cv2.destroyAllWindows()


/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_47653/2191231186.py:15: DeprecationWarning: setOutputDepth() is deprecated, the output is auto-enabled if used.
  stereo.setOutputDepth(True)
/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_47653/2191231186.py:16: DeprecationWarning: setConfidenceThreshold() is deprecated, Use 'initialConfig.setConfidenceThreshold()' instead
  stereo.setConfidenceThreshold(200)


[2024-04-22 21:42:51.467] [depthai] [warning] setOutputDepth is deprecated. The output is auto-enabled if used


/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_47653/2191231186.py:39: DeprecationWarning: Device(pipeline) starts the pipeline automatically. Use Device() and startPipeline(pipeline) otherwise
  device.startPipeline()


KeyboardInterrupt: 

In [2]:
import numpy as np
import cv2
import depthai as dai

def find_homography(src_pts, dst_pts):
    """
    Estimate homography matrix using RANSAC algorithm.
    """
    assert len(src_pts) >= 4 and len(dst_pts) >= 4, "At least 4 corresponding points are required."

    # Number of iterations for RANSAC
    num_iterations = 1000
    max_inliers = 0
    best_homography = None

    for _ in range(num_iterations):
        # Randomly select 4 points
        indices = np.random.choice(len(src_pts), 4, replace=False)
        src_sample = src_pts[indices]
        dst_sample = dst_pts[indices]

        # Compute homography matrix
        homography, _ = cv2.findHomography(src_sample, dst_sample)

        # Calculate inliers using current homography
        src_transformed = cv2.perspectiveTransform(src_pts.reshape(-1, 1, 2), homography)
        dst_diff = np.linalg.norm(src_transformed - dst_pts, axis=-1)
        inliers = np.sum(dst_diff < 3)  # Threshold for inliers

        # Update best homography if more inliers found
        if inliers > max_inliers:
            max_inliers = inliers
            best_homography = homography

    return best_homography

# Initialize variables for accumulating frames
stitched_image = None
prev_frame = None

# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a color camera node
camRgb = pipeline.createColorCamera()
camRgb.setPreviewSize(640, 480)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create output
xoutRgb = pipeline.createXLinkOut()
xoutRgb.setStreamName("rgb")
camRgb.preview.link(xoutRgb.input)

# Pipeline defined, now we can connect to the device
with dai.Device(pipeline) as device:
    # Start pipeline
    device.startPipeline()

    # Output queue will be used to get the rgb frames from the output defined above
    qRgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    # Capture first frame
    prev_frame = None
    stitched_image = None

    # Main loop
    while True:
        # Get RGB frames
        inRgb = qRgb.tryGet()
        if inRgb is not None:
            frame = inRgb.getCvFrame()
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # If previous frame exists, stitch the current frame with the previous one
            if prev_frame is not None:
                # Step 1: Detect features and compute descriptors
                orb = cv2.ORB_create()
                keypoints1, descriptors1 = orb.detectAndCompute(prev_frame, None)
                keypoints2, descriptors2 = orb.detectAndCompute(frame_gray, None)

                # Step 2: Match features between the two images
                matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                matches = matcher.match(descriptors1, descriptors2)

                # Step 3: Sort matches by their distance
                matches = sorted(matches, key=lambda x: x.distance)

                # Step 4: Check if enough matches are found
                if len(matches) < 4:
                    print("Not enough matches found to calculate homography")
                    continue

                # Step 5: Extract matched keypoints
                src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
                dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

                # Step 6: Estimate homography matrix
                homography = find_homography(src_pts, dst_pts)

                # Step 7: Warp previous frame to current frame
                height, width = prev_frame.shape[:2]
                result = cv2.warpPerspective(prev_frame, homography, (width * 2, height))

                # Step 8: Copy current frame to the result image
                result[:frame_gray.shape[0], :frame_gray.shape[1]] = frame_gray

                # Update stitched image
                stitched_image = result

            # Update previous frame
            prev_frame = frame_gray

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cv2.destroyAllWindows()

# Display the stitched image after the loop
if stitched_image is not None and stitched_image.shape[0] > 0 and stitched_image.shape[1] > 0:
    cv2.imshow('Stitched Image', stitched_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


/var/folders/z0/xlrh1vdx3qn_jfzx_nhpjt4h0000gn/T/ipykernel_47653/3995722637.py:59: DeprecationWarning: Device(pipeline) starts the pipeline automatically. Use Device() and startPipeline(pipeline) otherwise
  device.startPipeline()


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/core/src/batch_distance.cpp:275: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'batchDistance'
